In [83]:
import pathlib
import sys
import warnings
import xarray as xr
import numpy as np
import pandas as pd


In [69]:
wrf_hydro_file = '/Users/afox/Jedi/whj/wrf_hydro_nwm_jedi/test/Data/wrf_hydro_nwm_files/geometry_nwm_long_range_snow.nc'
target = xr.open_dataset(wrf_hydro_file)
target

<xarray.Dataset>
Dimensions:  (south_north: 24, west_east: 30, soil_layers_stag: 4, feature_id: 189)
Dimensions without coordinates: south_north, west_east, soil_layers_stag, feature_id
Data variables:
    HGT      (south_north, west_east) float32 ...
    XLAT     (south_north, west_east) float32 ...
    XLONG    (south_north, west_east) float32 ...
    ZS       (soil_layers_stag) float32 ...
    Length   (feature_id) float32 ...
    lat      (feature_id) float32 ...
    lon      (feature_id) float32 ...
Attributes: (12/21)
    domain_dir:             /home/vagrant/jedi/domains/private/taylor_park_v2_1
    config:                 nwm_long_range_snow
    hrldas_patch_filename:  hrldas_namelist_patches.json
    hydro_patch_filename:   hydro_namelist_patches.json
    lsm_dx:                 1000.0
    lsm_dy:                 1000.0
    ...                     ...
    stream_dx_name:         Length
    stream_xdim_name:       feature_id
    stream_lat_name:        lat
    stream_lon_name:        lon
    stream_src_file:        /home/vagrant/jedi/domains/private/taylor_park_v2...
    stream_src_md5:         b43e1fa8443092671bf1338556152f04

In [70]:

infile =  "/Users/afox/Jedi/ncldasv2_internal/ucldasv2/bundle/ucldasv2/test/Data/20x20x4/ucldasv2_gridspec.nc"

In [71]:
wrfinput_ds = xr.open_dataset(infile)
wrfinput_ds

<xarray.Dataset>
Dimensions:    (xaxis_1: 20, yaxis_1: 20, zaxis_1: 4, Time: 1)
Coordinates:
  * xaxis_1    (xaxis_1) float64 1.0 2.0 3.0 4.0 5.0 ... 17.0 18.0 19.0 20.0
  * yaxis_1    (yaxis_1) float64 1.0 2.0 3.0 4.0 5.0 ... 17.0 18.0 19.0 20.0
  * zaxis_1    (zaxis_1) float64 1.0 2.0 3.0 4.0
  * Time       (Time) float64 1.0
Data variables:
    lonh       (Time, xaxis_1) float64 ...
    lath       (Time, yaxis_1) float64 ...
    lon        (Time, yaxis_1, xaxis_1) float64 ...
    lat        (Time, yaxis_1, xaxis_1) float64 ...
    elevation  (Time, yaxis_1, xaxis_1) float64 ...
    mask2d     (Time, yaxis_1, xaxis_1) float64 ...
    nzo_zstar  (Time) float64 ...
    h          (Time, zaxis_1, yaxis_1, xaxis_1) float64 ...
    h_zstar    (Time, zaxis_1, yaxis_1, xaxis_1) float64 ...

In [72]:
wrfinput_ds=wrfinput_ds.rename({'yaxis_1': 'south_north', 'xaxis_1': 'west_east', 'zaxis_1':'soil_layers_stag'})
wrfinput_ds=wrfinput_ds.rename(name_dict={'lat': 'LAT', 'lon': 'XLONG', 'lat': 'XLAT', 'elevation':'HGT'})
wrfinput_ds

<xarray.Dataset>
Dimensions:           (west_east: 20, south_north: 20, soil_layers_stag: 4, Time: 1)
Coordinates:
  * west_east         (west_east) float64 1.0 2.0 3.0 4.0 ... 18.0 19.0 20.0
  * south_north       (south_north) float64 1.0 2.0 3.0 4.0 ... 18.0 19.0 20.0
  * soil_layers_stag  (soil_layers_stag) float64 1.0 2.0 3.0 4.0
  * Time              (Time) float64 1.0
Data variables:
    lonh              (Time, west_east) float64 270.8 271.9 ... 288.4 289.3
    lath              (Time, south_north) float64 39.28 40.1 ... 54.18 55.03
    XLONG             (Time, south_north, west_east) float64 270.8 ... 293.6
    XLAT              (Time, south_north, west_east) float64 39.28 ... 50.28
    HGT               (Time, south_north, west_east) float64 0.0 0.0 ... 367.4
    mask2d            (Time, south_north, west_east) float64 0.0 0.0 ... 1.0 1.0
    nzo_zstar         (Time) float64 4.0
    h                 (Time, soil_layers_stag, south_north, west_east) float64 ...
    h_zstar           (Time, soil_layers_stag, south_north, west_east) float64 ...

In [73]:
wrfinput_vars = [vv for vv in wrfinput_ds.variables]
wrfinput_vars

['west_east',
 'south_north',
 'soil_layers_stag',
 'Time',
 'lonh',
 'lath',
 'XLONG',
 'XLAT',
 'HGT',
 'mask2d',
 'nzo_zstar',
 'h',
 'h_zstar']

In [74]:
wrfinput_keep_vars = ['XLAT', 'XLONG', 'HGT']
wrfinput_drop_list = list(
    set(wrfinput_vars).difference(wrfinput_keep_vars))
lsm_geom = (wrfinput_ds.drop_vars(wrfinput_drop_list).squeeze('Time'))
lsm_geom

<xarray.Dataset>
Dimensions:  (south_north: 20, west_east: 20)
Dimensions without coordinates: south_north, west_east
Data variables:
    XLONG    (south_north, west_east) float64 270.8 271.9 272.9 ... 292.4 293.6
    XLAT     (south_north, west_east) float64 39.28 39.18 39.07 ... 50.65 50.28
    HGT      (south_north, west_east) float64 0.0 0.0 0.0 ... 478.7 200.9 367.4

In [75]:
lsm_attrs = {}
lsm_attrs['lsm_xdim_name'] = 'west_east'
lsm_attrs['lsm_ydim_name'] = 'south_north'
lsm_attrs['lsm_zdim_name'] = 'soil_layers_stag'
lsm_attrs['lsm_lat_name'] = 'XLAT'
lsm_attrs['lsm_lon_name'] = 'XLONG'
lsm_attrs['lsm_z_name'] = 'ZS'
lsm_attrs['lsm_sfc_elev_name'] = 'HGT'
lsm_attrs['lsm_src_file'] = 'Offline'

lsm_attrs['stream_dx_name'] = 'Length'
lsm_attrs['stream_xdim_name'] = 'feature_id'
lsm_attrs['stream_lat_name'] = 'lat'
lsm_attrs['stream_lon_name'] = 'lon'
lsm_attrs['stream_src_file'] = 'dummy'

lsm_geom.attrs = lsm_attrs
geom = lsm_geom

In [76]:

geom = geom.assign(Length = target.Length)
geom = geom.assign(lat = target.lat)
geom = geom.assign(lon = target.lon)
geom = geom.assign(ZS = target.ZS)
geom

<xarray.Dataset>
Dimensions:  (south_north: 20, west_east: 20, feature_id: 189, soil_layers_stag: 4)
Dimensions without coordinates: south_north, west_east, feature_id, soil_layers_stag
Data variables:
    XLONG    (south_north, west_east) float64 270.8 271.9 272.9 ... 292.4 293.6
    XLAT     (south_north, west_east) float64 39.28 39.18 39.07 ... 50.65 50.28
    HGT      (south_north, west_east) float64 0.0 0.0 0.0 ... 478.7 200.9 367.4
    Length   (feature_id) float32 2.65e+03 3.359e+03 ... 1.189e+03 1.363e+03
    lat      (feature_id) float32 39.05 39.04 39.03 39.02 ... 39.05 39.07 39.07
    lon      (feature_id) float32 -106.7 -106.6 -106.8 ... -106.5 -106.5 -106.5
    ZS       (soil_layers_stag) float32 0.05 0.25 0.7 1.5
Attributes: (12/13)
    lsm_xdim_name:      west_east
    lsm_ydim_name:      south_north
    lsm_zdim_name:      soil_layers_stag
    lsm_lat_name:       XLAT
    lsm_lon_name:       XLONG
    lsm_z_name:         ZS
    ...                 ...
    lsm_src_file:       Offline
    stream_dx_name:     Length
    stream_xdim_name:   feature_id
    stream_lat_name:    lat
    stream_lon_name:    lon
    stream_src_file:    dummy

In [77]:
out_file = '/Users/afox/Jedi/whj/wrf_hydro_nwm_jedi/test/Data/wrf_hydro_nwm_files/test.nc'
geom.to_netcdf(out_file)

In [78]:
wrf_hydro_rs = '/Users/afox/Jedi/whj/build/wrf_hydro_nwm_jedi/test/Data/wrf_hydro_nwm_files/RESTART.2017010100_DOMAIN1'
rs_temp = xr.open_dataset(wrf_hydro_rs)
rs_temp.Times

<xarray.DataArray 'Times' (Time: 1)>
array([b'2017-01-01_00:00:00'], dtype='|S19')
Dimensions without coordinates: Time

In [79]:
offline_rs = '/Users/afox/Jedi/ncldasv2_internal/ucldasv2/test/Data/20x20x4/INPUT/lnd.res.nc'
rs_offline = xr.open_dataset(offline_rs)
rs_offline


<xarray.Dataset>
Dimensions:  (time: 1, yaxis_1: 20, xaxis_1: 20)
Coordinates:
  * time     (time) float64 1.0
Dimensions without coordinates: yaxis_1, xaxis_1
Data variables:
    weasd    (time, yaxis_1, xaxis_1) float64 ...
    snwdph   (time, yaxis_1, xaxis_1) float64 ...

In [80]:
rs_offline = rs_offline.assign(Times = rs_temp.Times)
rs_offline

<xarray.Dataset>
Dimensions:  (time: 1, yaxis_1: 20, xaxis_1: 20, Time: 1)
Coordinates:
  * time     (time) float64 1.0
Dimensions without coordinates: yaxis_1, xaxis_1, Time
Data variables:
    weasd    (time, yaxis_1, xaxis_1) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    snwdph   (time, yaxis_1, xaxis_1) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    Times    (Time) |S19 b'2017-01-01_00:00:00'

In [81]:
out_file = '/Users/afox/Jedi/whj/build/wrf_hydro_nwm_jedi/test/Data/wrf_hydro_nwm_files/lnd.res.nc'
rs_offline.to_netcdf(out_file)
rs_offline

<xarray.Dataset>
Dimensions:  (time: 1, yaxis_1: 20, xaxis_1: 20, Time: 1)
Coordinates:
  * time     (time) float64 1.0
Dimensions without coordinates: yaxis_1, xaxis_1, Time
Data variables:
    weasd    (time, yaxis_1, xaxis_1) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    snwdph   (time, yaxis_1, xaxis_1) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    Times    (Time) |S19 b'2017-01-01_00:00:00'

In [117]:
init_file = '/Users/afox/Jedi/whj/ufs-land_C96_da_init_fields_1hr.nc'
static_file = '/Users/afox/Jedi/whj/ufs-land_C96_da_static_fields.nc'
restart_file = '/Users/afox/Jedi/whj/ufs_land_restart.2000-01-10_23-00-00.nc'

init_ds = xr.open_dataset(init_file)
static_ds = xr.open_dataset(static_file)
restart_ds = xr.open_dataset(restart_file)

In [121]:
lat = init_ds['latitude']
LAT = lat.values.reshape(449,1)
long = init_ds['longitude']
LONG = long.values.reshape(449,1)
hgt = static_ds['elevation']
HGT = hgt.values.reshape(449,1)

ds = xr.Dataset(
     {"XLAT": (("south_north", "west_east"), LAT),
     "XLONG": (("south_north", "west_east"), LONG),
     "HGT": (("south_north", "west_east"), HGT),
     },
     coords={
          "south_north": np.arange(1,450),
          "west_east": [1],
     },
     )

ds = ds.assign(ZS = target.ZS)
ds = ds.assign(Length = target.Length)
ds = ds.assign(lat = target.lat)
ds = ds.assign(lon = target.lon)

lsm_attrs = {}
lsm_attrs['lsm_xdim_name'] = 'west_east'
lsm_attrs['lsm_ydim_name'] = 'south_north'
lsm_attrs['lsm_zdim_name'] = 'soil_layers_stag'
lsm_attrs['lsm_lat_name'] = 'XLAT'
lsm_attrs['lsm_lon_name'] = 'XLONG'
lsm_attrs['lsm_z_name'] = 'ZS'
lsm_attrs['lsm_sfc_elev_name'] = 'HGT'
lsm_attrs['lsm_src_file'] = 'Offline'

lsm_attrs['stream_dx_name'] = 'Length'
lsm_attrs['stream_xdim_name'] = 'feature_id'
lsm_attrs['stream_lat_name'] = 'lat'
lsm_attrs['stream_lon_name'] = 'lon'
lsm_attrs['stream_src_file'] = 'dummy'

ds.attrs = lsm_attrs

geom_outfile = '/Users/afox/Jedi/whj/build/wrf_hydro_nwm_jedi/test/Data/wrf_hydro_nwm_files/offline_geometry.nc'
ds.to_netcdf(geom_outfile)

ds

<xarray.Dataset>
Dimensions:      (south_north: 449, west_east: 1, soil_layers_stag: 4, feature_id: 189)
Coordinates:
  * south_north  (south_north) int64 1 2 3 4 5 6 7 ... 444 445 446 447 448 449
  * west_east    (west_east) int64 1
Dimensions without coordinates: soil_layers_stag, feature_id
Data variables:
    XLAT         (south_north, west_east) float32 49.4 49.6 ... 33.07 32.21
    XLONG        (south_north, west_east) float32 279.4 278.2 ... 279.9 279.9
    HGT          (south_north, west_east) float32 287.1 232.8 ... 18.76 4.335
    ZS           (soil_layers_stag) float32 0.05 0.25 0.7 1.5
    Length       (feature_id) float32 2.65e+03 3.359e+03 ... 1.189e+03 1.363e+03
    lat          (feature_id) float32 39.05 39.04 39.03 ... 39.05 39.07 39.07
    lon          (feature_id) float32 -106.7 -106.6 -106.8 ... -106.5 -106.5
Attributes: (12/13)
    lsm_xdim_name:      west_east
    lsm_ydim_name:      south_north
    lsm_zdim_name:      soil_layers_stag
    lsm_lat_name:       XLAT
    lsm_lon_name:       XLONG
    lsm_z_name:         ZS
    ...                 ...
    lsm_src_file:       Offline
    stream_dx_name:     Length
    stream_xdim_name:   feature_id
    stream_lat_name:    lat
    stream_lon_name:    lon
    stream_src_file:    dummy

In [128]:
restart_ds = restart_ds.assign(Times = rs_temp.Times)

sneqv = restart_ds['snwdph']
SNEQV = sneqv.values.reshape(449,1)
SNEQV = sneqv.values.reshape(449,1)
temp_ds = xr.Dataset({"SNEQV": (("south_north", "west_east"), SNEQV)})

restart_ds = restart_ds.assign(SNEQV = temp_ds.SNEQV)

restart_outfile = '/Users/afox/Jedi/whj/build/wrf_hydro_nwm_jedi/test/Data/wrf_hydro_nwm_files/offline_restart.nc'
restart_ds.to_netcdf(restart_outfile)

temp_ds

<xarray.Dataset>
Dimensions:  (south_north: 449, west_east: 1)
Dimensions without coordinates: south_north, west_east
Data variables:
    SNEQV    (south_north, west_east) float64 219.8 139.7 180.6 ... 0.0 0.0 0.0

In [133]:
hofx_file = '/Users/afox/Jedi/whj/build/wrf_hydro_nwm_jedi/test/Data/hofx/hofx3d_offline.swe_0000.nc' 
hofx_ds = xr.open_dataset(hofx_file)
hofx_ds

<xarray.Dataset>
Dimensions:  (nlocs: 100)
Coordinates:
  * nlocs    (nlocs) int32 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0
Data variables:
    *empty*
Attributes:
    _ioda_layout:          ObsGroup
    _ioda_layout_version:  0